## Setup

In [90]:
import os
import pandas as pd

In [2]:
DATA_DIR = "../proc_data"
DATA_FILE = os.path.join(DATA_DIR, "processed.csv")

## Data Read

In [3]:
df = pd.read_csv(DATA_FILE)

In [4]:
df.shape

(170161, 11)

In [5]:
df.head()

,index,id,url,title,text,score,created_utc,subreddit,num_comments,upvote_ratio,over_18
0,189,1fxmtei,https://www.reddit.com/r/ucla/comments/1fxmtei...,Can I email back consulting clubs to ask for a...,My resume/application got rejected from a cons...,1,2024-10-06 18:10:40,ucla,0,1.00,False
1,190,1fxms88,https://i.redd.it/hvmfotrcf6td1.png,Got 100 days of free coffee! 🥲,A local Dunkin was giving away these away to t...,1,2024-10-06 18:09:19,DunkinDonuts,0,1.00,False
2,191,1fxmrv9,https://www.reddit.com/r/AffiliateCommunity/co...,30 percent commission! Easy sell,I’d love to invite you to join the AVY Coffee ...,1,2024-10-06 18:08:53,AffiliateCommunity,0,1.00,False
3,192,1fxmrnq,https://www.reddit.com/r/relationship_advice/c...,I (M28) confessed to my long time friend (F28)...,We’ve been friends for over 10 years. She brok...,1,2024-10-06 18:08:37,relationship_advice,1,1.00,False
4,193,1fxmr5d,https://www.reddit.com/r/CasualIreland/comment...,Is there a way to avoid having dinner with my ...,I don't like having dinner with my mam!! For s...,1,2024-10-06 18:08:00,CasualIreland,0,0.67,False


In [9]:
df.shape, df.drop_duplicates(subset='id').shape, df.drop_duplicates(subset='title').shape

((170161, 11), (170161, 11), (163512, 11))

## Data Model

- Posts table
- Subreddit table

#### Total number of posts in total

In [12]:
print("Total Posts: ", df.shape[0])

Total Posts:  170161


#### Most popular date

In [20]:
df['created_utc'] = pd.to_datetime(df['created_utc'])

In [21]:
df['created_date'] = df['created_utc'].dt.date

In [24]:
df_pop_date = df.groupby('created_date', as_index=False).agg(total_posts=('id', 'count'))

In [26]:
df_pop_date[df_pop_date.total_posts == df_pop_date.total_posts.max()]

,created_date,total_posts
122,2025-02-02,1483


#### Avg. posts per day

In [30]:
df_pop_date.total_posts.mean()

1242.051094890511

In [33]:
df_pop_date.sort_values(by='total_posts', ascending=True).iloc[1:].total_posts.mean()

1249.0514705882354

#### Top subreddit

In [34]:
df_pop_sub = df.groupby('subreddit', as_index=False).agg(total_posts=('id', 'count'))

In [37]:
df_pop_sub.sort_values(by='total_posts', ascending=False).head(10)

,subreddit,total_posts
15938,espresso,4567
19487,pourover,2367
5974,IndiaCoffee,1922
7344,MakeNewFriendsHere,1529
15387,dating_advice,1440
221,AITAH,1243
18726,nespresso,1189
19867,relationship_advice,1176
9165,PhR4Friends,1127
20688,starbucks,1015


In [35]:
df_pop_sub[df_pop_sub.total_posts == df_pop_sub.total_posts.max()]

,subreddit,total_posts
15938,espresso,4567


In [96]:
df_pop_sub.sort_values(by='total_posts', ascending=False).head(30)

,subreddit,total_posts
15938,espresso,4567
19487,pourover,2367
5974,IndiaCoffee,1922
7344,MakeNewFriendsHere,1529
15387,dating_advice,1440
221,AITAH,1243
18726,nespresso,1189
19867,relationship_advice,1176
9165,PhR4Friends,1127
20688,starbucks,1015


#### Stock Tracker

In [55]:
df_stock = df.groupby(['created_date', 'subreddit'], as_index=False) \
                .agg(total_posts=('id', 'count'))
df_stock['created_date'] = pd.to_datetime(df_stock['created_date'])
df_stock.sort_values(by=['created_date', 'total_posts'], ascending=[True, False], inplace=True)

In [56]:
df_stock.head(4)

,created_date,subreddit,total_posts
579,2024-10-03,espresso,36
6,2024-10-03,AITAH,16
110,2024-10-03,CoffeePH,15
217,2024-10-03,IndiaCoffee,14


In [52]:
# get previous day's posts
df_stock['prev_total_posts'] = df_stock.groupby('subreddit')['total_posts'].shift(1)

# compute percentage change
df_stock['pct_change'] = ((df_stock["total_posts"] - df_stock["prev_total_posts"]) / df_stock["prev_total_posts"])

In [53]:
df_stock.tail(10)

,created_date,subreddit,total_posts,prev_total_posts,pct_change
116494,2025-02-16,urbanfantasy,1,1.0,0.0
116495,2025-02-16,vegas,1,1.0,0.0
116496,2025-02-16,vegaslocals,1,1.0,0.0
116497,2025-02-16,venting,1,2.0,-0.5
116498,2025-02-16,wawacult,1,1.0,0.0
116499,2025-02-16,whatsthatbook,1,2.0,-0.5
116500,2025-02-16,wizardposting,1,1.0,0.0
116501,2025-02-16,women,1,1.0,0.0
116502,2025-02-16,woodworking,1,1.0,0.0
116503,2025-02-16,yoga,1,1.0,0.0


In [70]:
def compute_pct_change(df_stock, time_freq="D"):
    """
    Computes percentage change in total posts for different time frequencies.

    Args:
    df_stock (pd.DataFrame): DataFrame with 'created_date', 'subreddit', and 'total_posts'.
    time_freq (str): Frequency to aggregate data. Options:
        - 'D' for daily
        - 'W' for weekly
        - 'M' for monthly
        - 'Q' for quarterly
        - 'Y' for yearly

    Returns:
    pd.DataFrame: DataFrame with percentage change compared to the previous period.
    """
    # Ensure the date column is a datetime type
    df_stock["created_date"] = pd.to_datetime(df_stock["created_date"])

    # Define time grouping
    time_mapping = {
        "D": df_stock["created_date"],
        "W": df_stock["created_date"] - pd.to_timedelta(df_stock["created_date"].dt.weekday, unit="D"),
        "M": df_stock["created_date"].dt.to_period("M").dt.to_timestamp(),
        "Q": df_stock["created_date"].dt.to_period("Q").dt.to_timestamp(),
        "Y": df_stock["created_date"].dt.to_period("Y").dt.to_timestamp(),
    }

    if time_freq not in time_mapping:
        raise ValueError("Invalid time frequency. Choose from 'D', 'W', 'M', 'Q', 'Y'.")

    # Assign the correct time period
    df_stock["time_period"] = time_mapping[time_freq]

    # Aggregate total posts for each period and subreddit
    df_agg = df_stock.groupby(["time_period", "subreddit"], as_index=False)["total_posts"].sum()

    # Sort values to ensure correct shifting
    df_agg = df_agg.sort_values(by=["subreddit", "time_period"])

    # Compute previous period's total_posts
    df_agg["prev_total_posts"] = df_agg.groupby("subreddit")["total_posts"].shift(1)

    # Compute percentage change
    df_agg["pct_change"] = ((df_agg["total_posts"] - df_agg["prev_total_posts"]) / df_agg["prev_total_posts"]) * 100

    # Handle cases where previous period's posts were 0 (to avoid division by zero)
    df_agg["pct_change"] = df_agg["pct_change"].replace([float("inf"), -float("inf")], None).fillna(0)

    return df_agg

In [74]:
df_stock_month = compute_pct_change(df_stock, time_freq='M')

/var/folders/6p/b8fsbws114377d5kxxk54hdm0000gn/T/ipykernel_30877/2623520175.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_agg["pct_change"] = df_agg["pct_change"].replace([float("inf"), -float("inf")], None).fillna(0)


In [76]:
df_stock_month[df_stock_month.subreddit=='pourover']

,time_period,subreddit,total_posts,prev_total_posts,pct_change
19487,2024-10-01,pourover,405.0,NaN,0.000000
41720,2024-11-01,pourover,434.0,405.0,7.160494
63953,2024-12-01,pourover,585.0,434.0,34.792627
86186,2025-01-01,pourover,674.0,585.0,15.213675
108419,2025-02-01,pourover,269.0,674.0,-60.089021


In [92]:
df_stock_month[(df_stock_month['total_posts'] > 100) & (df_stock_month['pct_change'] > 50.0)]

,time_period,subreddit,total_posts,prev_total_posts,pct_change
70191,2025-01-01,DesignMyRoom,192.0,115.0,66.956522
50723,2024-12-01,JamesHoffmann,126.0,69.0,82.608696
51898,2024-12-01,Market76,222.0,19.0,1068.421053
38408,2024-11-01,findommes,107.0,45.0,137.777778
60641,2024-12-01,findommes,233.0,107.0,117.757009
83976,2025-01-01,interiordecorating,148.0,97.0,52.577320


In [95]:
df_stock_month[(df_stock_month['total_posts'] > 70) & (df_stock_month['pct_change'] < -50.0)]

,time_period,subreddit,total_posts,prev_total_posts,pct_change
89153,2025-02-01,AITAH,113.0,282.0,-59.929078
47224,2024-12-01,CoffeePH,81.0,350.0,-76.857143
92424,2025-02-01,DesignMyRoom,92.0,192.0,-52.083333
94906,2025-02-01,IndiaCoffee,214.0,485.0,-55.876289
96276,2025-02-01,MakeNewFriendsHere,175.0,385.0,-54.545455
56077,2024-12-01,TemuThings,74.0,315.0,-76.507937
104319,2025-02-01,dating_advice,183.0,404.0,-54.702970
104870,2025-02-01,espresso,529.0,1183.0,-55.283178
107658,2025-02-01,nespresso,134.0,323.0,-58.513932
108419,2025-02-01,pourover,269.0,674.0,-60.089021
